# 📊 Analyse NBA depuis 1950
Projet Data Science - De la Donnée à la Décision

## 1. 📥 Acquisition des Données

In [7]:
! pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\afitenudaeklou\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

seasons = pd.read_csv('data/Seasons_Stats.csv')
players = pd.read_csv('data/Players.csv')

## 2. 🧼 Nettoyage et Préparation

In [30]:
# Suppression des colonnes inutiles
seasons = seasons.drop(columns=['blanl', 'blank2'], errors='ignore')

# Suppression des lignes sans année ou joueur
seasons = seasons.dropna(subset=['Year', 'Player'])

# Conversion de l'année en entier
seasons['Year'] = seasons['Year'].astype(int)

# Calcul des points par match
seasons['PTS_per_game'] = seasons['PTS'] / seasons['G']

# Création de la colonne décennie (Era)
seasons['Era'] = (seasons['Year'] // 10) * 10

# Fusion avec le fichier Players pour ajouter height/weight/etc.
players = players.drop(columns=['Unnamed: 0'], errors='ignore')
df = pd.merge(seasons, players, on="Player", how="left")


## 3. 🔍 Analyse exploratoire et visualisation


In [12]:
! pip install nbformat --upgrade


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\afitenudaeklou\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
import nbformat
print(nbformat.__version__)


5.10.4


In [31]:
# Moyenne des points par décennie
era_pts = df.groupby('Era')['PTS_per_game'].mean().reset_index()
fig = px.bar(era_pts, x='Era', y='PTS_per_game', title='Évolution des points/match par décennie')
fig.show()


In [32]:
# Moyenne des points par poste
pos_pts = df.groupby('Pos')['PTS_per_game'].mean().reset_index().sort_values(by='PTS_per_game', ascending=False)
fig = px.bar(pos_pts, x='PTS_per_game', y='Pos', orientation='h', title='Moyenne PTS par poste')
fig.show()


In [18]:
import plotly.io as pio
pio.renderers.default = "notebook"  # ou "notebook_connected"


In [20]:
pio.renderers.default = "browser"
fig.show()


## 4. 📈 Visualisations

## 5. 🤖 Modélisation

In [33]:
# On encode la position (Pos) en valeur numérique
df['Pos_encoded'] = df['Pos'].astype('category').cat.codes

# Sélection des variables explicatives
features = ['Age', 'height', 'weight', 'MP', 'FGA', 'FTA', '3PA', 'Pos_encoded']

# On ne garde que les lignes sans valeurs manquantes
model_data = df[features + ['PTS_per_game']].dropna()

# Définir X (features) et y (target)
X = model_data[features]
y = model_data['PTS_per_game']

# Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraînement du modèle
model = LinearRegression().fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Évaluation du modèle
print("✅ R2 score :", r2_score(y_test, y_pred))


✅ R2 score : 0.863041799687897


In [34]:
# Affichage des coefficients
coef_df = pd.DataFrame({
    'Variable': features,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', ascending=False)

print(coef_df)


      Variable  Coefficient
0          Age     0.059487
7  Pos_encoded     0.029103
5          FTA     0.014312
4          FGA     0.014265
2       weight     0.010684
6          3PA     0.003173
3           MP    -0.002242
1       height    -0.007663
